In [1]:
import pandas as pd
import numpy as np
from sklearn.ensemble import BaggingRegressor, RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split
from sklearn import metrics
# pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

In [2]:
pre_train = pd.read_csv('train.csv')
# pre_train['parking_price'] = pre_train['parking_price'].fillna(int(0))
# pre_train['parking_area'] = pre_train['parking_area'].fillna(int(0))
# pre_train['txn_floor'] = pre_train['txn_floor'].fillna(int(0))
pre_train = pre_train.drop(['lat','lon'], axis=1)

In [3]:
dummy_fields = ['building_material', 'city', 'total_floor', 'building_type', 
                'building_use', 'parking_way', 'txn_floor', 'town']

for each in dummy_fields:
    dummies = pd.get_dummies( pre_train.loc[:, each], prefix=each ) 
    dum = pd.concat([pre_train, dummies], axis = 1 )

In [4]:
dataset = dum.drop(dummy_fields, axis = 1 )
dataset.head()

,building_id,txn_dt,building_complete_dt,parking_area,parking_price,land_area,building_area,village_income_median,town_population,town_area,town_population_density,doc_rate,master_rate,bachelor_rate,jobschool_rate,highschool_rate,junior_rate,elementary_rate,born_rate,death_rate,marriage_rate,divorce_rate,village,N_50,N_500,N_1000,N_5000,N_10000,I_10,I_50,I_index_50,I_100,I_250,I_500,I_index_500,I_1000,I_index_1000,I_5000,I_index_5000,I_10000,I_index_10000,I_MIN,II_10,II_50,II_index_50,II_100,II_250,II_500,II_index_500,II_1000,II_index_1000,II_5000,II_index_5000,II_10000,II_index_10000,II_MIN,III_10,III_50,III_index_50,III_100,III_250,III_500,III_index_500,III_1000,III_index_1000,III_5000,III_index_5000,III_10000,III_index_10000,III_MIN,IV_10,IV_50,IV_index_50,IV_100,IV_250,IV_500,IV_index_500,IV_1000,IV_index_1000,IV_5000,IV_index_5000,IV_10000,IV_index_10000,IV_MIN,V_10,V_50,V_index_50,V_100,V_250,V_500,V_index_500,V_1000,V_index_1000,V_5000,V_index_5000,V_10000,V_index_10000,V_MIN,VI_10,VI_50,VI_index_50,VI_100,VI_250,VI_500,VI_index_500,VI_1000,VI_index_1000,VI_5000,VI_index_5000,VI_10000,VI_index_10000,VI_MIN,VII_10,VII_50,VII_index_50,VII_100,VII_250,VII_500,VII_index_500,VII_1000,VII_index_1000,VII_5000,VII_index_5000,VII_10000,VII_index_10000,VII_MIN,VIII_10,VIII_50,VIII_index_50,VIII_100,VIII_250,VIII_500,VIII_index_500,VIII_1000,VIII_index_1000,VIII_5000,VIII_index_5000,VIII_10000,VIII_index_10000,VIII_MIN,IX_10,IX_50,IX_index_50,IX_100,IX_250,IX_500,IX_index_500,IX_1000,IX_index_1000,IX_5000,IX_index_5000,IX_10000,IX_index_10000,IX_MIN,X_10,X_50,X_index_50,X_100,X_250,X_500,X_index_500,X_1000,X_index_1000,X_5000,X_index_5000,X_10000,X_index_10000,X_MIN,XI_10,XI_50,XI_index_50,XI_100,XI_250,XI_500,XI_index_500,XI_1000,XI_index_1000,XI_5000,XI_index_5000,XI_10000,XI_index_10000,XI_MIN,XII_10,XII_50,XII_index_50,XII_100,XII_250,XII_500,XII_index_500,XII_1000,XII_index_1000,XII_5000,XII_index_5000,XII_10000,XII_index_10000,XII_MIN,XIII_10,XIII_50,XIII_index_50,XIII_100,XIII_250,XIII_500,XIII_index_500,XIII_1000,XIII_index_1000,XIII_5000,XIII_index_5000,XIII_10000,XIII_index_10000,XIII_MIN,XIV_10,XIV_50,XIV_index_50,XIV_100,XIV_250,XIV_500,XIV_index_500,XIV_1000,XIV_index_1000,XIV_5000,XIV_index_5000,XIV_10000,XIV_index_10000,XIV_MIN,total_price,town_0,town_4,town_6,town_7,town_8,town_9,town_10,town_11,town_12,town_13,town_15,town_17,town_18,town_19,town_20,town_22,town_23,town_25,town_26,town_27,town_29,town_30,town_31,town_32,town_34,town_38,town_39,town_40,town_41,town_42,town_43,town_44,town_46,town_48,town_49,town_50,town_52,town_53,town_54,town_59,town_61,town_62,town_63,town_64,town_66,town_74,town_76,town_79,town_80,town_84,town_87,town_89,town_90,town_91,town_94,town_96,town_97,town_101,town_102,town_103,town_105,town_107,town_108,town_109,town_110,town_111,town_112,town_113,town_115,town_118,town_119,town_120,town_121,town_122,town_123,town_126,town_127,town_128,town_130,town_131,town_132,town_134,town_136,town_139,town_141,town_142,town_143,town_144,town_146,town_148,town_151,town_152,town_154,town_155,town_156,town_157,town_158,town_160,town_161,town_163,town_164,town_165,town_166,town_167,town_169,town_172,town_174,town_175,town_177,town_179,town_180,town_181,town_182,town_184,town_185,town_186,town_187,town_188,town_191,town_192,town_193,town_194,town_195,town_196,town_197,town_198,town_199,town_204,town_205,town_206,town_207,town_208,town_210,town_211,town_214,town_215,town_216,town_217,town_218,town_221,town_222,town_223,town_224,town_225,town_227,town_228,town_232,town_234,town_236,town_238,town_239,town_240,town_243,town_244,town_248,town_249,town_250,town_252,town_253,town_254,town_255,town_258,town_259,town_261,town_263,town_267,town_268,town_269,town_271,town_272,town_273,town_274,town_275,town_279,town_280,town_282,town_283,town_284,town_285,town_289,town_290,town_292,town_295,town_296,town_299,town_300,town_301,town_302,town_304,town_307,town_311,town_312,town_315,town_316,town_317,town_320,town_322

In [5]:
dataset_dropna = dataset.dropna(axis=0)
dataset_dropna.shape

(3078, 438)

In [6]:
X = dataset_dropna.drop(['total_price', 'building_id'], axis=1)
y = dataset_dropna['total_price']

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
y_train, y_test = y_train.astype('int'), y_test.astype('int')

In [8]:
# np.isnan(X_train.values.all())

# a = np.isnan(X_dropna).any()
# print(a)

In [9]:
X_train = X_train.values.astype('float32')

In [10]:
np.isfinite(X_train.all())

True

In [11]:
X_train.shape

(2462, 436)

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
random_grid

In [ ]:
rf = RandomForestRegressor()
        # Random search of parameters, using 3 fold cross validation, 
        # search across 100 different combinations, and use all available cores
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 5, cv = 3, 
                               verbose=2, random_state=42, n_jobs = -1)
        # Fit the random search model
rf_random.fit(X_train, y_train)

In [ ]:
rf_random.best_params_

In [12]:
regressor_trees = RandomForestRegressor(n_estimators = 1400, min_samples_split = 5,
                                        min_samples_leaf = 1, max_features = 'sqrt',
                                        max_depth = 30, bootstrap = True)
regressor_trees.fit(X_train, y_train)
trees_pred = regressor_trees.predict(X_test)

In [13]:
y_test_np = y_test.values
y_test_np[0]

10116885

In [24]:
(trees_pred[2] - y_test_np[2]) / y_test_np[2]

0.06194617623506842

In [26]:
hit_rate = []
mape = []
for i in range(0, len(trees_pred)):
    mape.append((trees_pred[i] - y_test_np[i]) / y_test_np[i])
    if (trees_pred[i] - y_test_np[i]) / y_test_np[i] <= 0.1:
        hit_rate.append(int(1))
    else:
        hit_rate.append(int(0))
hit_rate_all = sum(np.array(hit_rate)) / len(trees_pred)
mape_all = sum(np.array(mape)) / len(trees_pred)

In [27]:
score = hit_rate_all*10000 + (1-mape_all)
score

4156.360750846954